In [9]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

In [10]:
df = pd.read_csv('Live_Usage_queries_with_mql_processed_rev3_curated.csv')

In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/llama-2-7b-chat-hf"

# The instruction dataset to use
#dataset_name = ""

# Fine-tuned model name
new_model = "llama-2-7b-fine-tuned-peft-02"

In [3]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [4]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [5]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/output"

# Number of training epochs
num_train_epochs = 100

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True


# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = 10

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 10

In [6]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [7]:
prompt_template = """### _LUMIN_ Question:
{question}

### Context:
{context}

### _LUMIN_ Answer:
{answer}
"""

In [ ]:
context = "Above _LUMIN_ Question is converted into below _LUMIN_ Answer. This has to be strictly converted into LUMIN context only."

In [12]:
df.columns

Index(['query', 'template_query_fuzzy'], dtype='object')

In [11]:
def create_fine_tuning_dataset(row):
    question = row['query']
    answer = row['template_query_fuzzy']
    formated = prompt_template.format(question=question,context=context,
                                      answer=answer)
    data_point = str({'question': formated, "answer":answer})
    return data_point

In [13]:
df['fine_tuning_dataset']=df.apply(create_fine_tuning_dataset, axis=1)

In [14]:
df.drop(columns=['query','template_query_fuzzy'], inplace=True)

In [15]:
dataset = Dataset.from_pandas(df)

In [16]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [17]:
compute_dtype

torch.float16

In [18]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [19]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" 

In [21]:
# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [24]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=500,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [25]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="fine_tuning_dataset",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map:   0%|          | 0/2543 [00:00<?, ? examples/s]

In [26]:
dataset[0]

{'fine_tuning_dataset': "{'question': '### _LUMIN_ Question:\\nwhy profit change in july 2022\\n\\n### Context:\\nThis is to be converted into LUMIN context\\n\\n### _LUMIN_ Answer:\\nWhy profit changed in july 2022\\n', 'answer': 'Why profit changed in july 2022'}"}

In [27]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.109600
20,1.009300
30,0.363000
40,0.302700
50,0.476400
60,0.309300
70,0.153700
80,0.085100
90,0.145200
100,0.242100


TrainOutput(global_step=500, training_loss=0.21054272067546845, metrics={'train_runtime': 2378.5899, 'train_samples_per_second': 0.841, 'train_steps_per_second': 0.21, 'total_flos': 3764844362661888.0, 'train_loss': 0.21054272067546845, 'epoch': 0.79})

In [28]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
from peft import PeftModel, PeftConfig

In [29]:
model_to_merge = PeftModel.from_pretrained(
    model,
    'llama-2-7b-fine-tuned-peft-02')

In [33]:
context = "This is to be converted into LUMIN context"

In [30]:
prompt_template = """### _LUMIN_ Question:
{question}

### Context:
{context}

"""

In [34]:
def predict_template_query(user_query):
    _input = prompt_template.format(question=user_query, context=context)
    _inputs = tokenizer.encode(_input, return_tensors="pt")
    outputs = model_to_merge.generate(input_ids=_inputs.to('cuda'), max_length= 200)
    output = tokenizer.decode(outputs[0])
    #print(output)
    a = output.split("_LUMIN_ Answer",1)[1]
    return a.split("\n")[1]

In [35]:
inp = 'why sales decrease in 4th quarter 2023'
predict_template_query(inp)

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2415: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


'Why sales changed in quarter4 2023'

In [36]:
inp = "why production dropped for bikes in 1st quarter 2022"
predict_template_query(inp)

'Why production of bikes changed in quarter1 2022'

In [37]:
inp = 'why sales of texas increase in 1st quarter 2023'
predict_template_query(inp)

'Why sales of texas changed in quarter1 2023'

In [ ]:
inp = 'drivers of profit'
predict_template_query(inp)